In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *


# Catalog Name
catalog = "workspace"

# Key Cols List
key_cols = "['passenger_id']"
key_cols_list = eval(key_cols)

# CDC Column
cdc_col ='processed_time'

# Backdated Refresh
backdated_refreshdated_refresh = ""

# Source Object
source_object = "silver_passenger"

# Source Schema
source_schema = "silver"

# Target Schema 
target_schema = "gold"

# Target Object 
target_object = "DimPassengers"

# Surrogate Key
surrogate_key = "DimPassengersKey"

print(key_cols_list)

# Catalog Name
catalog = "workspace"

# Key Cols List
key_cols = "['flight_id']"
key_cols_list = eval(key_cols)

# CDC Column
cdc_col = "processed_time"

# Backdated Refresh
backdated_refresh = ""

# Source Object
source_object = "silver_flight"

# Source Schema
source_schema = "silver"

# Target Schema 
target_schema = "gold"

# Target Object 
target_object = "DimFlights"

# Surrogate Key
surrogate_key = "DimFlightsKey"

In [0]:
# Catalog Name
catalog = "workspace"

# Key Cols List
key_cols = "['airport_id']"
key_cols_list = eval(key_cols)

# CDC Column
cdc_col = "processed_time"

# Backdated Refresh
backdated_refresh = ""

# Source Object
source_object = "silver_airport"

# Source Schema
source_schema = "silver"

# Target Schema 
target_schema = "gold"

# Target Object 
target_object = "DimAirports"

# Surrogate Key
surrogate_key = "DimAirportsKey"

#  Incremental Data

In [0]:
%sql
select * from workspace.silver.silver_airport

In [0]:
spark.sql(f"select max({cdc_col}) as lastloaded_time from workspace.{source_schema}.{source_object}").collect()

In [0]:
if len(backdated_refreshdated_refresh)==0:

  if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):
    last_loaded_date=spark.sql(f"""
                              select max({cdc_col}) as lastloaded_time 
                              from workspace.{target_schema}.{target_object} 
                              """).collect()[0][0]
  else:
    last_loaded_date= "1900-01-01 00:00:00"
else:
  last_loaded_date=backdated_refreshdated_refresh

In [0]:
print(last_loaded_date)


In [0]:
df_src=spark.sql(f"""
                 select * from workspace.{source_schema}.{source_object}
                 where {cdc_col}>'{last_loaded_date}'
                 """)

In [0]:
df_src.display()

In [0]:
if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):
    key_col_incremental=",".join(key_cols_list)

    df_trg=spark.sql(f"""
                  select {key_col_incremental},{surrogate_key}, create_date, update_date 
                      FROM {catalog}.{target_schema}.{target_object}
                  """)

else:
    key_cols_string_init = [f"'' AS {i}" for i in key_cols_list]
    key_cols_string_init = ", ".join(key_cols_string_init)
    df_trg=spark.sql(f"""
                     select {key_cols_string_init}, CAST('0' AS INT) AS {surrogate_key}, CAST('1900-01-01 00:00:00' AS timestamp) AS          create_date, CAST('1900-01-01 00:00:00' AS timestamp) AS update_date WHERE 1=0
                     """)

In [0]:
df_trg.display()

In [0]:
df_src.createOrReplaceTempView("source")
df_trg.createOrReplaceTempView("target")

join_condition = ' AND '.join([f"source.{i} = target.{i}" for i in key_cols_list])

df_join = spark.sql(f"""
            SELECT source.*, 
                   target.{surrogate_key},
                   target.create_date,
                   target.update_date
            FROM source
            LEFT JOIN target
            ON {join_condition}
            """)

In [0]:
df_join.display()

In [0]:
# OLD RECORDS
df_old = df_join.filter(col(f'{surrogate_key}').isNotNull())

# NEW RECOERDS
df_new = df_join.filter(col(f'{surrogate_key}').isNull())

In [0]:
df_old.display()

# Preparing old records

In [0]:
df_old_enr = df_old.withColumn('update_date', current_timestamp())

# New Records

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"): 
    max_surrogate_key = spark.sql(f"""
                            SELECT max({surrogate_key}) FROM {catalog}.{target_schema}.{target_object}
                        """).collect()[0][0]
    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key)+lit(1)+monotonically_increasing_id())\
                    .withColumn('create_date', current_timestamp())\
                    .withColumn('update_date', current_timestamp())    

else:
    max_surrogate_key = 0
    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key)+lit(1)+monotonically_increasing_id())\
                    .withColumn('create_date', current_timestamp())\
                    .withColumn('update_date', current_timestamp())

In [0]:
df_new_enr.display()

## Union

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)

## Upsert

In [0]:
from delta.tables import DeltaTable 
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_union.alias("src"), f"trg.{surrogate_key} = src.{surrogate_key}")\
                        .whenMatchedUpdateAll(condition = f"src.{cdc_col} >= trg.{cdc_col}")\
                        .whenNotMatchedInsertAll()\
                        .execute()

else: 

    df_union.write.format("delta")\
            .mode("append")\
            .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

In [0]:
%sql
select * from  workspace.gold.dimflights